In [ ]:
from skimage.measure import regionprops_table
from skimage.transform import rescale,resize
import pandas as pd
from laptrack import LapTrack
import numpy as np
import napari

# Load images and show in napari

In [ ]:
viewer = napari.Viewer()
data = np.load("data/data_small.npz")
images=data["images"]
labels=data["labels"]

viewer.add_image(images)
labels_layer=viewer.add_labels(labels)

# Summarize regionprops

In [ ]:
regionprops = []
for frame, label in enumerate(labels):
    df = pd.DataFrame(
        regionprops_table(label,properties=["label","centroid"])    
    )
    df["frame"]=frame
    regionprops.append(df)
regionprops_df = pd.concat(regionprops)

# Track centroids by LapTrack

In [ ]:
lt = LapTrack(
    track_cost_cutoff=15**2,
    splitting_cost_cutoff=30**2
)
track_df, split_df, _ = lt.predict_dataframe(
    regionprops_df.copy(),
    coordinate_cols=["centroid-0","centroid-1"],
    only_coordinate_cols=False,
    )
track_df = track_df.reset_index()
viewer.add_tracks(
    track_df[["track_id","frame","centroid-0","centroid-1"]]
)

# Show clonal cells by the same colors

In [ ]:
new_labels = np.zeros_like(labels)

for i, row in track_df.iterrows():
    frame=int(row["frame"])
    inds = labels[frame] == row["label"]
    new_labels[frame][inds] = int(row["tree_id"])+1
viewer.add_labels(new_labels)
labels_layer.visible=False